In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 2 - 2.2 Cascata

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.flagCartaoCreditoContratado':[0],
                             'solicitante.flagLimiteGarantidoContratado':[0],
                             'payloadHomol.intermediarias.clienteBlindado':[0],
                             'payloadHomol.intermediarias.clienteBloqueado':[0],
                             'payloadHomol.intermediarias.serasaInvalido':[0],
                             'payloadHomol.saidas.mensagemFiltro':['99. Accepted'],
                             'payloadHomol.intermediarias.semInfoScore':[0],
                             'solicitante.rendaLiquidaPicpayBatch':[10000]
                            })

In [0]:
#Criação das massas com casos forçados
#Digito CPF Concomitante CP
massa_Cascata_1 = sherlock.Gera_Massa(base_entrada)
massa_Cascata_1.gera_cenario('payloadHomol.saidas.mensagemFiltro',["-99999",
                                                           "02. Cadastro Irregular",
                                                           "03. Cadastro Negado ou Em Andamento",
                                                           "04. Conta Inativa ou Conta Teste PicPay",
                                                           "05. Sistema Operacional Defasado",
                                                           "06. CPF Invalido",
                                                           "07. Obito",
                                                           "09. Fraude PicPay",
                                                           "10. Chargeback PicPay",
                                                           "11. Idade <= 20 ou >= 80",
                                                           "12. Atraso PicPay",
                                                           "13. Atraso Original",
                                                           "14. Reneg ou FNV Ativo",
                                                           "15. Valor de Restritivo BVS >= 10% da Renda",
                                                           "16. Valor de Restritivo Serasa >= 10% da Renda",
                                                           "17. Quantidade de IFs = 0",
                                                           "18. Atraso Bacen >= 10% da Renda",
                                                           "19. Prejuizo Bacen >= 10% da Renda",
                                                           "20. Reneg Bacen",
                                                           "21. Saldo Financiado de Cartao > 70% do Limite",
                                                           "22. IU Cheque > 90%",
                                                           "23. AD Bacen",
                                                           "24. Desenrola",
                                                           "25. No hit Bureaus",
                                                           "99. Accepted"
                                                           ])
massa_Cascata_1 = massa_Cascata_1.get_massa_final()

massa_Cascata_2 = sherlock.Gera_Massa(base_entrada)
massa_Cascata_2.gera_cenario('solicitante.flagCartaoCreditoContratado',[0,1]) #2
massa_Cascata_2.gera_cenario('solicitante.flagLimiteGarantidoContratado',[0,1]) #4
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.clienteBlindado',[0,1]) #8
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.clienteBloqueado',[0,1]) #16
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.serasaInvalido',[0,1]) #32
massa_Cascata_2.gera_cenario('payloadHomol.intermediarias.semInfoScore',[0,1]) #64
massa_Cascata_2.gera_cenario('solicitante.rendaLiquidaPicpayBatch',[1000,-99999,0,600]) #256
massa_Cascata_2.gera_cenario('payloadHomol.saidas.mensagemFiltro',["-99999","24. Desenrola","25. No hit Bureaus","99. Accepted"]) # 1024
massa_Cascata_2 = massa_Cascata_2.get_massa_final()

#Empilha todos casos com 1 caso padrão.
massa_CascataFinal = sherlock.Gera_Massa(base_entrada)
massa_CascataFinal.empilha_massas([massa_Cascata_1,
                                   massa_Cascata_2
])
                        


massa_CascataFinal.set_bom_arqv()
massa_CascataFinal.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_CascataFinal.get_massa_json(),rp.FiltroVisaoPolitica)

In [0]:
# Lógica para validar, pode-se trocar o nome da variável na linha 6 para exibir uma freq dos dominios que essa variável possui.

import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['saidas']['mensagemFiltroPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Lógica para validar, pode-se trocar o nome da variável na linha 6 para exibir uma freq dos dominios que essa variável possui.

import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['passouRegraFiltroPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha = freqzinha.sort_values(by=['valor'], ascending=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_CascataFinal.rename_cabecalho()

# Adaptando alguns nomes de variaveis para a função de teste.
massa_CascataFinal.rename_variavel("payloadHomol.intermediarias.clienteBloqueado","clienteBloqueado")
massa_CascataFinal.rename_variavel("payloadHomol.intermediarias.clienteBlindado","clienteBlindado")
massa_CascataFinal.rename_variavel("flagCartaoCreditoContratado","payloadParam.solicitante.flagCartaoCreditoContratado")
massa_CascataFinal.rename_variavel("flagLimiteGarantidoContratado","payloadParam.solicitante.flagLimiteGarantidoContratado")
massa_CascataFinal.rename_variavel("rendaLiquidaPicpayBatch","payloadParam.solicitante.rendaLiquidaPicpayBatch")

In [0]:
massa_CascataFinal.atribui_expected(listaResultado,"payloadHomol.saidas.mensagemFiltroPolitica","saida.mensagemFiltroPolitica")
massa_CascataFinal.atribui_expected(listaResultado,"payloadHomol.saidas.statusDecisao","statusDecisao")

massa_CascataFinal.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_CascataFinal.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaFiltrosVisaoPolitica.csv',index=False)

In [0]:
# Análise Divergencias
data_id = 94
print(json.dumps(listaResultado[data_id-1],indent=4))

#